**The Data Science Method**


1.   [Problem Identification](https://medium.com/@aiden.dataminer/the-data-science-method-problem-identification-6ffcda1e5152)

2.   [Data Wrangling](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-data-collection-organization-and-definitions-d19b6ff141c4)
  * Data Collection - Collected data from wikipedia and quandl wiki price dataset. The wikipedia showed us the currect S&P 500 companies and used their ticker symbols to query quandl wiki prices.
  * Data Organization - Done using cookiecutter
  * Data Definition
  * Data Cleaning - The S&P 500 data from quandls wiki price is clean and ready for analysis use but has lost its support from Quandl community as of April 11, 2018. So we will use this dataset to setup the protfolio optimizer with proof of concept then use a different data source later for cost efficiencies.

3.   [**Exploratory Data Analysis**](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-exploratory-data-analysis-bc84d4d8d3f9)
 * Build data profile tables and plots
        - Outliers & Anomalies
 * Explore data relationships
 * Identification and creation of features

4.   [Pre-processing and Training Data Development](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967)
  * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   [Modeling](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b)
  * Create dummy or indicator features for categorical variable
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   [Documentation](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-documentation-c92c28bd45e6)

  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code
  * Finalize Documentation


First, loads the needed packages and modules into Python. Then loads the data into a pandas dataframe for ease of use.

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

from portopt.src.get_data import WikipediaData, SlickData, YahooFinanceData


import ipywidgets as widgets
%matplotlib inline



ModuleNotFoundError: No module named 'portopt.src.get_data'

In [2]:
# prints current directory
current_dir = os.getcwd()
print("Current Directory: ")
print(current_dir)

Current Directory: 
/Users/jb/Development/courses/springboard/ds/Assignments/Portfolio-Optimization/notebooks/exploratory


In [3]:
# prints parent directory
project_dir = os.path.abspath(os.path.join(os.path.join(current_dir, os.pardir), os.pardir))
print("Parent Directory: ")
print(project_dir)

Parent Directory: 
/Users/jb/Development/courses/springboard/ds/Assignments/Portfolio-Optimization


In [4]:
print(os.listdir())

['3.0-jujbates-S&P500-PO_eda.ipynb', '.DS_Store', '.ipynb_checkpoints', '2.0-jujbates-S&P500-PO_data_wrangling.ipynb']


In [5]:
plt.style.use('dark_background')
c = ['white', 'springgreen', 'fuchsia', 'lightcoral', 'red'] # Color
s = [24, 20, 16, 12]  # Size
w = [0.75, 1, 1.25, 1.50] # Line Width
ga = 0.10 # Grid Alpha

In [ ]:
Uses 

In [ ]:
sp_df.info()

In [ ]:
sp_slick_df

In [ ]:
sp_df.index

In [ ]:
sp_df.index = pd.to_datetime(sp_df.index)
sp_df.index

In [ ]:
sp_df.info()

In [ ]:
# def _plot_adj_close(ax, df, df_cols, ticker, title=None, ylabel=None):
    ax.set_title(title, fontsize=s[1])
    ax.set_ylabel(ylabel, fontsize=s[2])
    ax.plot(df.index.values, df.iloc[:,:100], linewidth=w[0], color=c[0], alpha=0.15)
    ax.plot(df.index.values, df[ticker], label=ticker, linewidth=w[3], color=c[2])
    
    ax.legend(loc='upper left', title = 'Tickers')
    ax.grid(alpha=ga)


def plot_sector_adj_close(sector, ticker):
    
    fig, (ax1) = plt.subplots(1, figsize=(20, 24))

    _plot_adj_close(ax1, sp_df, sp_df.loc[:,ticker], ticker,
                   f"S&P 500 Stocks ({sector} - {ticker})",
                   f"Price")
    
    plt.xticks(fontsize=s[3])
    plt.yticks(fontsize=s[3])
    plt.show()
    return 

interact(plot_sector_adj_close, 
         sector=sp_df, 
         ticker=sp_df);



In [ ]:

def plot_adj_close_daily_return_hist(ticker):
    
    sp_daily_lag_df = sp_df.shift(1)
    sp_dr_df = (sp_daily_lag_df/sp_df) - 1

    
    mean = sp_dr_df.mean()
    
    
    std = sp_dr_df.std()

    fig, (ax1, ax2) = plt.subplots(2, figsize=(20, 16))
    ax1.set_title(f"S&P 500 Stocks ({ticker})", fontsize=s[1])
    ax1.set_ylabel(f"Price", fontsize=s[2])
    ax1.plot(sp_df.index.values, sp_df[ticker], label=ticker, linewidth=w[3], color=c[2])
    
    ax1.legend(loc='upper left', title = 'Tickers')
    ax1.grid(alpha=ga)
    
    ax2.set_title(f"S&P 500 Stocks ({ticker})", fontsize=s[1])
    ax2.set_ylabel(f"Price", fontsize=s[2])
    
    ax2.hist(sp_dr_df[ticker], bins=50, label=ticker, linewidth=w[3], color=c[1])
    
    ax2.axvline(mean[ticker], color='red', linestyle='dashed', linewidth=2)
    #to plot the std line we plot both the positive and negative values 
    ax2.axvline(std[ticker], color='g', linestyle='dashed', linewidth=2)
    ax2.axvline(-std[ticker], color='g', linestyle='dashed', linewidth=2)

    ax2.legend(loc='upper left', title = 'Tickers')
    ax2.grid(alpha=ga)
    
    plt.xticks(fontsize=s[3])
    plt.yticks(fontsize=s[3])
    plt.show()
    
    return 

interact(plot_adj_close_daily_return_hist, 
         ticker=sp_df);



#### Cumulative Returns

In [ ]:
# Cumulative Return
def cumulative_return(df):
    return (df.iloc[-1,:] / df.iloc[0,:]) - 1

sp_cr = cumulative_return(sp_df)
sp_cr.head()

In [ ]:
# Calculate percentage returns
returns = sp_df.pct_change()

In [ ]:
# Calculate individual mean returns 
meanDailyReturns = returns.mean()

In [ ]:
# Define weights for the portfolio
weights = np.array([100 / sp_df.shape[1] for x in range(sp_df.shape[1])])

In [ ]:
# Calculate expected portfolio performance
portReturn = np.sum(meanDailyReturns * weights)

In [ ]:
# Create portfolio returns column
returns['Portfolio'] = returns.dot(weights)

In [ ]:
# Calculate cumulative returns
daily_cum_ret=(1+returns).cumprod()

In [ ]:
# Plot the portfolio cumulative returns only
# fig, ax = plt.subplots()
# ax.plot(daily_cum_ret.index, daily_cum_ret.Portfolio, color=c[2], label="Portfolio")
# plt.legend()
# plt.show()



fig, ax = plt.subplots(1, figsize=(20, 8))
ax.set_title("title", fontsize=s[1])
ax.set_ylabel("ylabel", fontsize=s[2])
ax.plot(daily_cum_ret.index, daily_cum_ret.Portfolio, color=c[2], label="Portfolio")

ax.legend(loc='upper right')
ax.grid(alpha=ga)

In [ ]:
# Get percentage daily returns
daily_returns = sp_df.pct_change()

# Calculate the covariance matrix 
cov_matrix = (daily_returns.cov())*252

# Calculate the portfolio variance
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))

# Print the result
print(str(np.round(portfolio_variance, 4) * 100) + '%')

# Calculate the standard deviation by taking the square root
portfolio_standard_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Print the results 
print(str(np.round(portfolio_standard_dev, 4) * 100) + '%')


In [ ]:
# Calculate total rate of return from start to end
total_return = (sp_df.iloc[-1,:] - sp_df.iloc[0,:]) / sp_df.iloc[0,:]
print(total_return)

In [ ]:
# Annualize return
annualized_return = ((1 + total_return)**(12 / 12)) - 1
annualized_return['Portfolio'] = annualized_return.dot(weights)
print (annualized_return)

In [ ]:
# risk-free rate
rfr = 0

# Calculate annualized volatility from the standard deviation
vol_sp500 = returns['Portfolio'].std() * np.sqrt(252)

# Calculate the Sharpe ratio 
sharpe_ratio = ((annualized_return['Portfolio'] - rfr) / vol_sp500)
print (sharpe_ratio)

In [ ]:

fig, (ax1, ax2) = plt.subplots(2, figsize=(20, 16))
ax1.set_title(f"S&P 500 Stocks Returns (Portfolio)", fontsize=s[1])
ax1.set_ylabel(f"Percentage of Change", fontsize=s[2])
ax1.plot(returns.index.values, returns.Portfolio*100, label='Portfolio', linewidth=w[3], color=c[2])

ax1.legend(loc='upper left')
ax1.grid(alpha=ga)


returns_mean = returns.mean()
returns_std = returns.std()

ax2.set_title(f"S&P 500 Stocks Returns Distribution (Portfolio)", fontsize=s[1])
ax2.set_ylabel(f"Frequency", fontsize=s[2])
ax2.set_xlabel(f"Percentage of Change", fontsize=s[2])

ax2.hist(returns.Portfolio*100, bins=50, label='Portfolio', linewidth=w[3], color=c[1])

ax2.axvline(returns_mean.Portfolio*100, color='red', linestyle='dashed', linewidth=2)
#to plot the std line we plot both the positive and negative values 
ax2.axvline(returns_std.Portfolio*100, color='g', linestyle='dashed', linewidth=2)
ax2.axvline(-returns_std.Portfolio*100, color='g', linestyle='dashed', linewidth=2)

ax2.legend(loc='upper left', title = 'Tickers')
ax2.grid(alpha=ga)

plt.xticks(fontsize=s[3])
plt.yticks(fontsize=s[3])
plt.show()


In [ ]:
returns_skew = returns.skew()
returns_kurtosis = returns.kurtosis()

print('Returns Skew: ')
print(returns_skew.Portfolio)
print('---------------------------------')
print('Returns Kurtosis: ')
print(returns_kurtosis.Portfolio)

In [ ]:
target = 0

# Create a downside return column with the negative returns only
downside_returns = returns.loc[returns.Portfolio < target]

# Calculate expected return and std dev of downside
expected_return_mean = returns.Portfolio.mean()
downside_returns_std = downside_returns.Portfolio.std()

# Calculate the sortino ratio
sortino_ratio = (expected_return_mean - rfr)/downside_returns_std

# Print the results
print("Expected return  : ", expected_return_mean*100)
print("Downside risk   : ", downside_returns_std*100)
print("Sortino ratio : ", sortino_ratio)

In [ ]:
# Calculate the max value 
roll_max = sp_df.rolling(center=False, min_periods=1, window=252).max()

# Calculate the daily draw-down relative to the max
daily_draw_down = sp_df/roll_max - 1.0

# Calculate the minimum (negative) daily draw-down
max_daily_draw_down = daily_draw_down.rolling(center=False, min_periods=1, window=252).min()





In [ ]:
def plot_daily_down(ticker):

    fig, ax1 = plt.subplots(1, figsize=(20, 16))
    ax1.set_title(f"S&P 500 Stocks ({ticker})", fontsize=s[1])
    ax1.set_ylabel(f"Price", fontsize=s[2])
    # Plot the results
    ax1.plot(sp_df.index.values, daily_draw_down[ticker], label='Daily drawdown')
    ax1.plot(sp_df.index.values, max_daily_draw_down[ticker], label='Maximum daily drawdown in time-window')

    ax1.legend(loc='upper right', title = 'Tickers')
    ax1.grid(alpha=ga)
    
    return 

interact(plot_daily_down, 
         ticker=sp_df);



In [ ]:
weights.sum()

In [ ]:
# Calculate return of the portfolio
# total_return_pf = (weights*portfolio_data['mean_return']).sum()